In [2]:
# Drone Position control with MPC
# ----------------------
# An optimal control problem (OCP),
# solved with direct multiple-shooting.

from casadi import *
import numpy as np

# Paper Cascaded Nonlinear MPC for Realtime Quadrotor Position Tracking. Schlagenhauf
# Paper Non-Linear Model Predictive Control Using CasADi Package for Trajectory Tracking of Quadrotor. Elhesasy

# 1) Define system, static/kinematic/dynamic model. Typically Differential (Algebraic) equations
# Expression graphs represent a computation in the computer memory. SX, MX

# Ansatz https://github.com/casadi/casadi/blob/70f5cc19a288a1e753d06670060d2563aa63fafa/docs/examples/python/rocket.py

opti = Opti()  # Optimization problem

Ix = 0.0000166  # Moment of inertia around p_WB_W_x-axis, source: Julian Förster's ETH Bachelor Thesis
Iy = 0.0000167  # Moment of inertia around p_WB_W_y-axis, source: Julian Förster's ETH Bachelor Thesis
Iz = 0.00000293  # Moment of inertia around p_WB_W_z-axis, source: Julian Förster's ETH Bachelor Thesis
m = 0.029  # mass of Crazyflie 2.1
g = 9.81

Nx = 12
Nu = 4
Nhoriz = 10

In [3]:
# ---- decision variables ---------

# x(t) = [x, y, z, phi, theta, psi, x_dot, y_dot, z_dot, phi_dot, ˙theta_dot, psi_dot]T
X = opti.variable(Nx, Nhoriz + 1)
U = opti.variable(Nu, Nhoriz)

xref = MX.zeros(Nx,1)
xref[2] = 1

Xref = repmat(xref, 1, Nhoriz + 1)

# repmat(v, n, m): Repeat expression v
# n times vertically and m times horizontally.
# repmat(SX(3), 2, 1) will create a 2 - by - 1 matrix with all elements 3.

x = MX.sym("x", Nx)
u = MX.sym("u", Nu)

# Ansatz: Nonlinear MPC tutorial with CasADi 3.5 https://www.youtube.com/watch?v=JI-AyLv68Xs&ab_channel=JorisGillis

x_pos = x[0]  # x-position
y = x[1]  # y-position
z = x[2]  # z-position
phi = x[3]  # phi-angle, Euler angles
theta = x[4]  # theta-angle, Euler angles
psi = x[5]  # psi-angle, Euler angles
x_pos_dot = x[6]  # x velocity
y_dot = x[7]  # y velocity
z_dot = x[8]  # z velocity
phi_dot = x[9]  # phi_dot, angular velocity
theta_dot = x[10]  # theta_dot
psi_dot = x[11]  # psi-dot

thrust = u[0]
tau_phi = u[1]
tau_theta = u[2]
tau_psi = u[3]

x_pos_ddot = (cos(phi) * sin(phi) * cos(psi) + sin(phi) * sin(phi)) * thrust / m
y_ddot = (cos(phi) * sin(phi) * cos(psi) - sin(phi) * sin(phi)) * thrust / m
z_ddot = -g + (cos(phi) * cos(theta)) * thrust / m
phi_ddot = theta_dot * psi_dot * (Iy - Iz) / Ix + tau_phi / Ix
theta_ddot = phi_dot * psi_dot * (Iz - Ix) / Iy + tau_theta / Iy
psi_ddot = theta_dot * phi_dot * (Ix - Iy) / Iz + tau_psi / Iz

# x˙(t) = [x_dot, y_dot, z_dot, phi_dot, ˙theta_dot, psi_dot, x_ddot, y_ddot, z_ddot, phi_ddot, ˙theta_ddot, psi_ddot]T
ode = vertcat(x_pos_dot, y_dot, z_dot, phi_dot, theta_dot, psi_dot, x_pos_ddot, y_ddot, z_ddot, phi_ddot, theta_ddot, psi_ddot)

In [4]:
# 2) Define problem based on system. Initial value/Integration/Rootfinding Problems/
# Nonlinear constrained optimization

# Expression graphs -create-> create functions

f = Function('f', [x, u], [ode], ['x', 'u'], ['x_dot'])

#f = lambda x,u: x_dot # f(x,u) = dx/dt  

f

Function(f:(x[12],u[4])->(x_dot[12]) MXFunction)

In [5]:
# Integrate with Explicit Euler over 0.2 seconds
dt = 0.01  # Time step
xj = x
for j in range(20):
  fj = f(xj,u)
  xj += dt*fj
  
# Discrete time dynamics function
F = Function('F', [x,u],[xj])

In [15]:
# ---- objective          ---------


Q = diag(MX([100, 100, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
R = diag(MX([10.0, 10.0, 10.0, 10.0]))

# Add the objective function to the optimization problem
objective = 0  # Initialize the objective function

# Loop over the control horizon
for k in range(Nhoriz):
    # State tracking cost
    state_error = X[:, k] - Xref[:, k]  # State deviation
    objective += state_error.T @ Q @ state_error  # Quadratic cost for state deviation

    # Control effort cost
    if k < Nhoriz - 1:  # No control input for the last stage
        control_error = U[:, k]  # Assuming zero as the reference for control inputs
        objective += control_error.T @ R @ control_error  # Quadratic cost for control effort

# Terminal cost
terminal_error = X[:, Nhoriz] - Xref[:, Nhoriz]  # Terminal state deviation
objective += terminal_error.T @ Q @ terminal_error  # Quadratic cost for terminal state deviation

# Solve/deploy problem
# Numerical backends, 3rd-party solvers
# Ipopt

opti.minimize(objective)  # Set the objective in the optimization problem